In [11]:
#Restart kernel if rerunning this cell
import os 
wd = os.path.dirname(os.getcwd()).replace('\\', '/')
os.chdir(wd)

import pandas as pd
import geopandas as gpd
import scripts.preprocess_helper as ph
import scripts.cleanup_helper as ch
import scripts.overlay_map_helper as omh
import omegaconf
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
yaml_path = os.path.join(wd, 'conf', 'config.yaml')
cfg = omegaconf.OmegaConf.load(yaml_path)
paths = cfg.paths
consts = cfg.constants

In [3]:
%%time
# Load the raw data
utils_raw = ph.load_data(paths.utilities.util_shape_file_path, 'shp')
j40_raw = ph.load_data(paths.j40.j40_shp_path, 'shp')
coal_raw = ph.load_data(paths.energy.coal_shp_path, 'shp')
ffe_raw = ph.load_data(paths.energy.ffe_shp_path, 'shp')
states_fips = ph.load_data(paths.boundaries.state_fips_path, 'csv')
ct_raw = ph.load_data(paths.boundaries.ct_path, 'shp')
st_raw = ph.load_data(paths.boundaries.st_path, 'shp')
dci_raw = ph.load_data(paths.dci.dci_csv_path, 'csv')
zip_shp = ph.load_data(paths.dci.zip_shp_path, 'shp')

CPU times: total: 3min 24s
Wall time: 9min 35s


In [4]:
%%time
#Data Cleaning using the helper files
util_clean, rural_coops, municipal_utils = ch.coops_utils_cleanup(util_shp = utils_raw, state_csv_df=states_fips, consts = consts.utilities)
j40_clean = ch.j40_cleanup(j40_shp = j40_raw.copy(), consts = consts.j40)
coal_clean, ffe_clean = ch.energy_cleanup(coal_shp = coal_raw.copy(), ffe_shp = ffe_raw.copy(), consts = consts.energy)
lic_clean = ch.low_inc_cleanup(pov_csv_path = paths.low_income.pov_dir, li_tract_csv_path= paths.low_income.tract_inc_dir, \
    li_st_csv_path=paths.low_income.st_inc_dir, li_msa_csv_path= paths.low_income.msa_inc_dir, \
        tracts_shp_path= paths.low_income.tract_shp_path ,msa_shp_path = paths.low_income.msa_shape_path , consts= consts.low_income)
county_df, state_df = ch.cty_st_borders_cleanup(county_df = ct_raw.copy(), st_fips_csv= states_fips, st_df = st_raw.copy(),\
     consts = consts.ct_st_borders)
dci_clean = ch.dci_cleanup(dci_csv_df = dci_raw.copy(), zip_shp = zip_shp.copy(), consts = consts.dci)

CPU times: total: 2min 3s
Wall time: 5min 58s


In [17]:
%%time
j40_coop_overlay = omh.overlays(community='j40', coops_utils= rural_coops, community_data = j40_clean)
coal_coop_overlay = omh.overlays(community='coal', coops_utils= rural_coops, community_data = coal_clean)
ffe_coop_overlay = omh.overlays(community='ffe', coops_utils= rural_coops, community_data = ffe_clean)
lic_coop_overlay = omh.overlays(community='low_income', coops_utils= rural_coops, community_data = lic_clean)
dci_coop_overlay = omh.overlays(community='dci', coops_utils= rural_coops, community_data = dci_clean)

j40_muni_overlay = omh.overlays(community='j40', coops_utils= municipal_utils, community_data = j40_clean)
coal_muni_overlay = omh.overlays(community='coal', coops_utils= municipal_utils, community_data = coal_clean)
ffe_muni_overlay = omh.overlays(community='ffe', coops_utils= municipal_utils, community_data = ffe_clean)
lic_muni_overlay = omh.overlays(community='low_income', coops_utils= municipal_utils, community_data = lic_clean)
dci_muni_overlay = omh.overlays(community='dci', coops_utils= municipal_utils, community_data = dci_clean)

CPU times: total: 1min 51s
Wall time: 4min 44s


In [20]:
dci_coop_overlay.head(2)

,ID,NAME,ADDRESS,CITY,ST_Code,ZIP,TELEPHONE,TYPE,COUNTRY,NAICS_CODE,NAICS_DESC,REGULATED,RETAIL_MWH,WSALE_MWH,TOTAL_MWH,TRANS_MWH,CUSTOMERS,YEAR,Shape__Are,Shape__Len,State,area,Type,zip_code,metro_area,city,county,state,state_abbr,region,cty_type,total_pop,dist_score,quintile,geoid,area_land,area_water,lat,lon,geometry,areaInters,perc_cover
0,10009,KARNES ELECTRIC COOP INC,1007 N. HWY. 123,KARNES CITY,TX,78118,(830) 780-3952,COOPERATIVE,USA,2211,"ELECTRIC POWER GENERATION, TRANSMISSION AND DI...",NOT AVAILABLE,1129899,-999999,1170464,0,21327,2020,3.085930,8.976781,Texas,4.364930e+10,Rural Co-Op,77954,NaN,"Cuero, TX","DeWitt County, Texas",Texas,TX,South,Nonmetro rural,"11,047",94.6,5,77954.0,981733668.0,1612017.0,+29.0964877,-097.2957161,"POLYGON ((-10846060.589 3406775.572, -10845194...",1.227909e+09,2.8131
1,1892,"BLUEBONNET ELECTRIC COOP, INC",650 HWY. 21 EAST,BASTROP,TX,78602,(888) 622-2583,COOPERATIVE,USA,2211,"ELECTRIC POWER GENERATION, TRANSMISSION AND DI...",REGULATED,2347639,-999999,2505121,0,106364,2020,2.814366,10.271066,Texas,4.031380e+10,Rural Co-Op,77954,NaN,"Cuero, TX","DeWitt County, Texas",Texas,TX,South,Nonmetro rural,"11,047",94.6,5,77954.0,981733668.0,1612017.0,+29.0964877,-097.2957161,"POLYGON ((-10843136.226 3409222.792, -10844332...",6.449297e+07,0.1600


In [ ]:
import folium

m = folium.Map(location=[39.8283, -98.5795], zoom_start=4, control_scale=True)
folium.GeoJson(dci_coop_overlay[dci_coop_overlay['state'] == 'Illinois']).add_to(m)
m